In [ ]:
import json

def load_json_file(file_path: str) -> dict:
    """
    Load and parse a JSON file
    
    Args:
        file_path (str): Path to the JSON file
        
    Returns:
        dict: Parsed JSON data as a dictionary
    """
    try:
        with open(file_path, 'r') as f:
            return json.load(f)
    except FileNotFoundError:
        print(f"Error: File {file_path} not found")
        return {}
    except json.JSONDecodeError:
        print(f"Error: Invalid JSON format in {file_path}")
        return {}

In [7]:
t={
    "is_new_session": False,
    "is_action": False,
    "endpoint": "/winlost_detail",
    "params": {
        "from_date": "2025-04-10",
        "to_date": "2025-04-10",
        "product": "All",
        "product_detail": "All",
        "level": "All",
        "user": "N/A"
    },
    "current_time": "2025-04-25 15:49:37"
}

t.pop("is_new_session")
t

{'is_action': False,
 'endpoint': '/winlost_detail',
 'params': {'from_date': '2025-04-10',
  'to_date': '2025-04-10',
  'product': 'All',
  'product_detail': 'All',
  'level': 'All',
  'user': 'N/A'},
 'current_time': '2025-04-25 15:49:37'}

In [14]:
import datetime

current_datetime = datetime.datetime.now()
expired_datetime = (current_datetime - datetime.datetime.strptime(t['current_time'], '%Y-%m-%d %H:%M:%S')).total_seconds() / 3600

In [15]:
expired_datetime

65.41141857527778

In [1]:
import re

def filter_top_pattern(text: str) -> bool:
    """
    Filter text containing 'top' or 'Top' pattern using regex
    
    Args:
        text (str): Input text to check
        
    Returns:
        bool: True if pattern found, False otherwise
    """
    pattern = r'[tT]op'
    return bool(re.search(pattern, text))


In [2]:
import requests
import json

def get_user_prompt(query: str) -> str:
    prompt = """
        User request: {query}

        # Example requests and responses:
        
        Input: "I need to see the win/loss report from last week"
        Output: {{
            "function_called": "/get_winlost_report"
        }}

        Input: "I want to get the reward report"
        Output: {{
            "function_called": "/get_reward_report"
        }}

        Input: "I want to take bet count report for user 123"
        Output: {{
            "function_called": "/get_betcount_report"
        }}

        Input: "Get me the get gross commission report for March transactions"
        Output: {{
            "function_called": "/get_gross_comm_report"
        }}

        Input: "I want get performance of abc1 last week"
        Output: {{
            "function_called": "N/A"
        }}
        
        Input: "Hello how are you today?"
        Output: {{
            "function_called": "N/A"
        }}
        
        Input: "I want Sportsbook only"
        Output: {{
            "function_called": "N/A"
        }}
        
        Input: "I want to get the top 20 outstanding"
        Output: {{
            "function_called": "/get_topoutstanding_report"
        }}
        
        Input: "The outstanding of Master1"
        Output: {{
            "function_called": "/get_outstanding_report"
        }}
        
        Input: "Top 40 Outstanding of Sportsbook"
        Output: {{
            "function_called": "/get_topoutstanding_report"
        }}
        
        Input: "My current outstanding"
        Output: {{
            "function_called": "/get_outstanding_report"
        }}
        
        Input: "Top 10 Outstanding of Sportsbook"
        Output: {{
            "function_called": "/get_topoutstanding_report"
        }}
        
        Based on this request, which function should be called? Return only the JSON response.
    """.format(query=query)
    
    return prompt


def functional_calling(query: str) -> str:
    # API endpoint
    url = "http://10.5.10.110:8090/api/chat"
    
    # Construct the system prompt using function metadata
    system_prompt = """
    You are an AI assistant that helps determine which function to call based on user input.
    Available functions:
        - /get_winlost_report: Get Win/Loss reports. Supports exporting reports (CSV/PDF), comparing trends across time periods, highlighting biggest winners/losers, and detecting unusual betting patterns. The Win Loss Report helps you easily track performance by showing either your own win/loss or your downline’s. Use it to review results and settle balances with your downline or upline efficiently.
        - /get_betcount_report: Get turnover/revenue reports showing total betting volume/activity. Includes trend analysis and comparison features.
        - /get_outstanding_report: The Outstanding Report helps you quickly track your current outstanding amount or the outstanding of a specific downline. You can also extract a list of top downlines with the highest outstanding for better monitoring and settlement planning.
        - /get_topoutstanding_report: The Top Outstanding Report helps you quickly track the top downlines with the highest outstanding. You can also extract a list of top downlines with the highest outstanding for better monitoring and settlement planning.

    Determine which function best matches the user's request and return it in JSON format like:
    {
        "function_called": "/function_name"
    }
    """

    # Construct the prompt for function determination with examples
    user_prompt = get_user_prompt(query)

    
    format_schema =  {
        "type": "object",
        "properties": {
            "function_called": {
                "type": "string",
                "description": "The name of the function to call",
                "enum": [
                    '/get_winlost_report',
                    '/get_turnover_report',
                    '/get_outstanding_report',
                    '/get_topoutstanding_report',
                    'N/A'
                ]
            }
        },
        "required": ["function_called"]
    }

    model = 'qwen2.5:14b'
    
    messages = [
        {"role": "system", "content": system_prompt},
        {"role": "user", "content": user_prompt}
    ]

    headers = {"Content-Type": "application/json"}

    data = {
        "model": model,
        "messages": messages,
        "format": format_schema,
        "stream": False
    }

    response = requests.post(url, headers=headers, data=json.dumps(data))
    return json.loads(response.json()['message']['content'])

In [ ]:

# Example usage
example_inputs = queries = [
    "Get me a Win Loss Detail Report yesterday",
    "Get me a Turnover Detail Report on day 10",
    "Net Turnover Detail Report from 1/1 to 31/1",
    "Get me a Win Loss Detail Report for Direct Member who played Product Detail Sportsbook in Sportsbook Product from 01/02/2024 to 15/02/2024",
    "Give me my Win Loss Detail report  last week",
    "Give me a Gross Commission Report Detail report for Sportsbook from last week.",
    "Give me my Win Loss Detail report  last week",
    "Get me a Company Report of master1",
    "Win/Loss details for Product Sportsbook",
    "Get me a Win Loss on day 10",
    "I want Win Loss report of Sportsbook this week",
    "I want get performance of abc1 last week",
    "Live Casino Master report of Master master1 yesterday",
    "I need Member report of Agent agent01 last week",
    "Show me today's summary",
    "How did we perform in live casino last week?",
    "The amount I need to pay for master abc1 last week",
    "Get report of master1",
    "Win/Loss details for Product Sportsbook",
    "Get me a Super Report of master1"
]

for input_text in example_inputs:
    called_function = functional_calling(input_text)
    print(f"Input: {input_text}")
    print(f"Called function: {called_function['function_called']}")
    print("\n")


In [2]:
functional_calling("Hello")

{'function_called': 'N/A'}

In [3]:
functional_calling("I want Sportsbook only")

{'function_called': 'N/A'}

In [3]:
functional_calling("give me top 20 outstanding")

{'function_called': '/get_topoutstanding_report'}

In [4]:
functional_calling("give me outstanding")

{'function_called': '/get_outstanding_report'}

In [5]:
functional_calling("I want to get the list of leading 20 outstanding")

{'function_called': '/get_topoutstanding_report'}

In [6]:
functional_calling("give me the first 20 outstanding sorting from highest to lowest")

{'function_called': '/get_topoutstanding_report'}

In [7]:
functional_calling("give me the first 20 outstanding decreasing")

{'function_called': '/get_topoutstanding_report'}

In [10]:
functional_calling("I want to get turnover report")

{'function_called': '/get_turnover_report'}

In [11]:
functional_calling("Do you know donald trump?")

{'function_called': 'N/A'}

In [12]:
functional_calling("How many outstanding do you have?")

{'function_called': '/get_outstanding_report'}